In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
data = gpd.read_file('Q3_VegCompPlots_Seward.shp')
print data.head()

   cat     site                       pttype      plotID                 uid  \
0    1  Council  Vegetation Composition Plot  CL-BEL-7-1  CL-VG-COMP-BEL-7-1   
1    2  Council  Vegetation Composition Plot    CL-BEL-2    CL-VG-COMP-BEL-2   
2    3  Council  Vegetation Composition Plot    CL-BEL-3    CL-VG-COMP-BEL-3   
3    4  Council  Vegetation Composition Plot    CL-BEL-4    CL-VG-COMP-BEL-4   
4    5  Council  Vegetation Composition Plot    CL-BEL-5    CL-VG-COMP-BEL-5   

        lat        lon        source                             vegtypev2  \
0  64.85751 -163.69321  Garmin eTrex  Birch-Ericaceous-lichen shrub tundra   
1  64.86078 -163.69829  Garmin eTrex  Birch-Ericaceous-lichen shrub tundra   
2  64.85934 -163.70839  Garmin eTrex  Birch-Ericaceous-lichen shrub tundra   
3  64.84800 -163.71053  Garmin eTrex  Birch-Ericaceous-lichen shrub tundra   
4  64.84961 -163.72963  Garmin eTrex  Birch-Ericaceous-lichen shrub tundra   

                              vegtypeL3           

In [2]:
# Create lookup table 
uniq_veg = data["vegtypeL3"].unique().tolist()
lookup_table = pd.DataFrame([],columns=["Code","Name"], index=np.arange(0,len(uniq_veg),1))
for i in range(0,len(lookup_table)):
    veg = uniq_veg[i]
    lookup_table["Name"][i] = veg
    uid = data[data["vegtypeL3"] == veg]["uid"].reset_index(drop=True)[0]
    lookup_table["Code"][i] = uid.split("-")[3]
print lookup_table

     Code                                  Name
0     BEL  Birch-Ericaceous-Lichen Shrub Tundra
1      TT                 Tussock-Lichen Tundra
2     WAC                     Wet Meadow Tundra
3    PMRB                  Wet Sedge Bog-Meadow
4      AS                    Alder-Willow Shrub
5   ELDST         Ericaceous Dwarf Shrub Tundra
6    DLST       Dryas-Lichen Dwarf Shrub Tundra
7     ASV      Mixed Shrub-Sedge Tussock Tundra
8     WBT                    Willow-Birch Shrub
9      MM           Mesic Graminoid-Herb Meadow
10   SWDT             Sedge-Willow-Dryas Tundra
11     MS                          Willow Shrub


In [3]:
bob = pd.read_csv("NGEEArctic_SewardPeninsula_Q3SamplingLocations_20191006_abreen - #2_Q3 Sampling points dGPS.csv", header=0)
bob = bob.iloc[0:164,:]
tf = bob["unique ID"].str.contains("VG-").dropna()
indices = tf[tf==True].index
bob = bob.iloc[indices,:]
bob = bob.reset_index(drop=True)
bob["unique ID"] = bob["unique ID"].replace("VG-COMP-","",regex=True)
bob["unique ID"] = bob["unique ID"].replace("_VG-COMP-","",regex=True)
bob["unique ID"] = bob["unique ID"].replace("VG-BIO-","",regex=True)
bob["unique ID"] = bob["unique ID"].replace("_","-",regex=True)

In [4]:
print bob

        site                   point type     location  \
0    Council      Vegetation Biomass Plot      CL-IS-2   
1    Council      Vegetation Biomass Plot      CL-IS-8   
2    Council      Vegetation Biomass Plot      CL-IS-3   
3    Council      Vegetation Biomass Plot      CL-IS-7   
4    Council      Vegetation Biomass Plot      CL-IS-5   
5    Council      Vegetation Biomass Plot      CL-IS-1   
6    Council      Vegetation Biomass Plot      CL-IS-6   
7    Council      Vegetation Biomass Plot      CL-IS-4   
8    Council  Vegetation Composition Plot      CL-IS-2   
9    Council  Vegetation Composition Plot          NaN   
10   Council  Vegetation Composition Plot          NaN   
11   Council  Vegetation Composition Plot          NaN   
12   Council  Vegetation Composition Plot      CL-IS-8   
13   Council  Vegetation Composition Plot          NaN   
14   Council  Vegetation Composition Plot          NaN   
15   Council  Vegetation Composition Plot          NaN   
16   Council  

In [84]:
output = pd.DataFrame([],columns=["Site","Veg","Northing","Easting"],index=np.arange(0,len(bob)*4))
site_dict = {"CL":"Council","KG":"Kougarok","TL":"Teller"}
direction = ["NE","NW","SE","SW"]
j = 0

for direc in direction:
  nor_col = direc + "_Plot_Corner_UTM_Northing"
  eas_col = direc + "_Plot_Corner_UTM_Easting"
  for i in range (0, len(bob)):
    site = bob["unique ID"][i].split("-")[0]
    veg = bob["unique ID"][i].split("-")[1]
    lis = lookup_table[lookup_table["Code"] == veg]["Name"].tolist()
    if len(lis) > 0:
      output["Site"][j] = site_dict[site]
      output["Veg"][j] = lis[0]
      output["Northing"][j] = bob[nor_col][i] 
      output["Easting"][j] = bob[eas_col][i]
      j+=1  
output.to_csv("additional_points.csv")

In [83]:
output.shape

(504, 4)